In [1]:
import pingouin as pg
import pandas as pd
import numpy as np
from scipy.stats import chi2
import warnings
warnings.filterwarnings('ignore')


In [42]:
# Load a dataset tailored for chi-squared independence tests from the Pingouin library.
# This particular dataset is widely used to demonstrate the application of statistical tests on categorical data, 
# especially in the context of medical or health-related research.
df = pg.read_dataset('chi2_independence')

# Transform the 'restecg' (resting electrocardiographic results) column values into strings,
# and append '_restecg' to each to ensure clear, distinct categorization.
# This modification aids in the analysis by making each category of electrocardiographic results easily identifiable.
df.restecg = df.restecg.astype(str) + '_restecg'

# Similarly, transform the 'cp' column values, which represent chest pain types, into strings,
# and append '_cp' to distinguish these categories clearly.
# This step enhances data readability, particularly for analyses focusing on chest pain types in medical research.
df.cp = df.cp.astype(str) + '_cp'

# Calculate and display the number of unique categories for 'restecg' and 'cp'.
# This information is vital for the planning of statistical analyses, such as chi-squared tests,
# which rely on the distribution across categories to test for independence or association.
unique_restecg_categories = df.restecg.nunique()
unique_cp_categories = df.cp.nunique()

print(f"Number of unique categories in 'restecg' (resting electrocardiographic results): {unique_restecg_categories}")
print(f"Number of unique categories in 'cp' (chest pain types): {unique_cp_categories}")

# Create a frequency table to examine the occurrence counts of each 'cp' (chest pain type) and 'restecg' (resting electrocardiographic result) combination.
# This frequency table is crucial for visualizing the distribution and potential correlations between the variables,
# offering insights into how different types of chest pain are associated with various electrocardiographic outcomes.
freq_table = pd.crosstab(df['cp'], df['restecg'])

# Display the frequency table to provide a quick overview of the data.
# Analyzing this table can help identify patterns or trends in the relationship between chest pain types and electrocardiographic results,
# which is important for medical research and diagnostic processes.
freq_table


Number of unique categories in 'restecg' (resting electrocardiographic results): 3
Number of unique categories in 'cp' (chest pain types): 4


restecg,0_restecg,1_restecg,2_restecg
cp,,,
0_cp,78,62,3
1_cp,19,31,0
2_cp,36,50,1
3_cp,14,9,0


In [43]:
import numpy as np
import pandas as pd
from scipy.stats import chi2

def calculate_ransacking_chi2(table,alpha=0.05):
    """
    Calculate the ransacking Chi-squared statistics for a 2x2 contingency table.

    Args:
    table (np.array): A 2x2 numpy array representing the contingency table.

    Returns:
    dict: A dictionary containing the odds ratio, log odds ratio, standard error, z-value,
          critical z-value, and the result of the hypothesis test.
    """
    # Calculate the odds for each group in the table
    odds_1 = table[0, 0] / table[0, 1]
    odds_2 = table[1, 0] / table[1, 1]

    # Calculate the odds ratio and its natural logarithm (log odds ratio)
    odds_ratio = odds_1 / odds_2
    G = np.log(odds_ratio)

    # Calculate the standard error of the log odds ratio
    SEG = np.sqrt(sum(1 / table.flatten()))

    # Compute the z-value for the log odds ratio
    z_value = G / SEG

    # Determine the critical z-value based on the desired confidence level (95% here)
    dof = 1
    chi_square_critical = chi2.ppf(1-alpha, dof)
    z_critical = np.sqrt(chi_square_critical)

    # Determine whether to reject the null hypothesis based on the z-value and critical z-value
    result = "reject" if abs(z_value) > z_critical else "fail to reject"

    return {
        "odds_ratio": odds_ratio,
        "log_odds_ratio_G": G,
        "standard_error_SEG": SEG,
        "z_value": z_value,
        "critical_z_value": z_critical,
        "result": result
    }

def chi_square_critical_value(dof, alpha=0.05):
    """
    Calculate the critical chi-square value for a given degrees of freedom and alpha level.

    Args:
    dof (int): Degrees of freedom.
    alpha (float): Significance level.

    Returns:
    float: The critical chi-square value.
    """
    # Calculate the critical value for the chi-square distribution
    critical_value = chi2.ppf(1 - alpha, dof)
    return np.round(np.sqrt(critical_value), 2)

def calculate_ransacking_chi2_for_each_cell(df, row_var, col_var, alpha=0.05, adjusted=False):
    """
    Calculates the ransacking chi-squared test for each cell of a contingency table formed
    by two categorical variables in a dataframe.

    Args:
    df (pd.DataFrame): The dataframe containing the data.
    row_var (str): The name of the row variable.
    col_var (str): The name of the column variable.
    adjusted (bool, optional): Whether to adjust the significance level for multiple tests. Defaults to False.

    Returns:
    pd.DataFrame: A dataframe with the ransacking chi-squared test results for each cell.
    """
    # Create a frequency table
    freq_table = pd.crosstab(df[row_var], df[col_var])
    total = freq_table.values.sum()
    num_tests = freq_table.shape[0] * freq_table.shape[1]
    adjusted_alpha = alpha / num_tests if adjusted else alpha
    dof = (freq_table.shape[0] - 1) * (freq_table.shape[1] - 1)

    results = []

    # Iterate through each cell in the frequency table to perform the test
    for row_label, col_label in freq_table.stack().index:
        cell_value = freq_table.loc[row_label, col_label]
        row_total = freq_table.loc[row_label].sum() - cell_value
        col_total = freq_table[col_label].sum() - cell_value
        remaining_total = total - cell_value - row_total - col_total

        table_2x2 = np.array([[cell_value, row_total], [col_total, remaining_total]])

        # Calculate ransacking chi2 values for the 2x2 table
        result = calculate_ransacking_chi2(table_2x2,alpha=alpha)

        # Calculate the adjusted critical Z-value for the significance level considering multiple tests
        adjusted_critical_z = chi_square_critical_value(dof, adjusted_alpha)

        # Determine the result based on the adjusted critical Z-value
        adjusted_result = "reject" if abs(result['z_value']) > adjusted_critical_z else "fail to reject"

        # Append the results for this cell to the list
        results.append({
            'Row': row_label,
            'Column': col_label,
            'Odds Ratio': result['odds_ratio'],
            'Log Odds Ratio (G)': result['log_odds_ratio_G'],
            'Standard Error (SEG)': result['standard_error_SEG'],
            'Z Value': result['z_value'],
            'Unadjusted Critical Z Value': result['critical_z_value'],
            'Adjusted Critical Z Value': adjusted_critical_z,
            'Unadjusted Result': result['result'],
            'Adjusted Result': adjusted_result,
            'Table': table_2x2
        })

    # Print critical Z-values for reference
    print(f"Critical Z : {result['critical_z_value']:.2f}")
    print(f"Adjusted Critical Z : {adjusted_critical_z:.2f}")

    # Return the results as a DataFrame for easy viewing
    return pd.DataFrame(results)

# Example usage (assuming 'df' is a DataFrame with 'cp' and 'restecg' columns):
ransacking_results_adjusted = calculate_ransacking_chi2_for_each_cell(df, 'cp', 'restecg', alpha=0.05, adjusted=True)
ransacking_results_adjusted.round(2)


Critical Z : 1.96
Adjusted Critical Z : 4.36


,Row,Column,Odds Ratio,Log Odds Ratio (G),Standard Error (SEG),Z Value,Unadjusted Critical Z Value,Adjusted Critical Z Value,Unadjusted Result,Adjusted Result,Table
0,0_cp,0_restecg,1.58,0.46,0.23,1.98,1.96,4.36,reject,fail to reject,"[[78, 65], [69, 91]]"
1,0_cp,1_restecg,0.60,-0.52,0.23,-2.23,1.96,4.36,reject,fail to reject,"[[62, 81], [90, 70]]"
2,0_cp,2_restecg,3.41,1.23,1.16,1.06,1.96,4.36,fail to reject,fail to reject,"[[3, 140], [1, 159]]"
3,1_cp,0_restecg,0.60,-0.51,0.32,-1.62,1.96,4.36,fail to reject,fail to reject,"[[19, 31], [128, 125]]"
4,1_cp,1_restecg,1.78,0.58,0.32,1.82,1.96,4.36,fail to reject,fail to reject,"[[31, 19], [121, 132]]"
5,1_cp,2_restecg,0.00,-inf,inf,NaN,1.96,4.36,fail to reject,fail to reject,"[[0, 50], [4, 249]]"
6,2_cp,0_restecg,0.67,-0.40,0.26,-1.57,1.96,4.36,fail to reject,fail to reject,"[[36, 51], [111, 105]]"
7,2_cp,1_restecg,1.51,0.41,0.26,1.61,1.96,4.36,fail to reject,fail to reject,"[[50, 37], [102, 114]]"
8,2_cp,2_restecg,0.83,-0.19,1.16,-0.16,1.96,4.36,fail to reject,fail to reject,"[[1, 86], [3, 213]]"
9,3_cp,0_restecg,1.72,0.54,0.44,1.22,1.96,4.36,fail to reject,fail to reject,"[[14, 9], [133, 147]]"


For an alpha of 0.05, the unadjusted ransacking result (based on Critical Z = +/- 1.96) found that only the relationships of 0_cp with 0_restecg and 1_restecg are significant.  When evaluating the Log Odds Ratios, since we do not know what the categories correspond to, we can indicate that this significance means 0_cp is positively related to 0_restecg and negatively related to 1_restecg. According to the adjusted results, none of the categories' relationships can be statistically associated.


In [45]:
pg.chi2_independence(df,x='cp',y='restecg')[2].round(2)
# Based on this outcome, the relationship between these two categories is not statistically significant; 
# let's evaluate them as subgroups from a categorical perspective.

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00,9.69,6.0,0.14,0.13,0.33
1,cressie-read,0.67,9.84,6.0,0.13,0.13,0.33
2,log-likelihood,0.00,10.57,6.0,0.10,0.13,0.36
3,freeman-tukey,-0.50,NaN,6.0,NaN,NaN,NaN
4,mod-log-likelihood,-1.00,inf,6.0,0.00,inf,NaN
5,neyman,-2.00,NaN,6.0,NaN,NaN,NaN


In [46]:
df = pg.read_dataset('chi2_independence')
subgroupdf = pd.get_dummies(df[['cp','restecg']],columns=['cp','restecg']).astype(int)
subgroupdf.head()

,cp_0,cp_1,cp_2,cp_3,restecg_0,restecg_1,restecg_2
0,0,0,0,1,1,0,0
1,0,0,1,0,0,1,0
2,0,1,0,0,1,0,0
3,0,1,0,0,0,1,0
4,1,0,0,0,0,1,0


In [47]:
pg.chi2_independence(subgroupdf,x='cp_0',y='restecg_0')[2].round(2)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00,3.50,1.0,0.06,0.11,0.46
1,cressie-read,0.67,3.50,1.0,0.06,0.11,0.46
2,log-likelihood,0.00,3.51,1.0,0.06,0.11,0.47
3,freeman-tukey,-0.50,3.51,1.0,0.06,0.11,0.47
4,mod-log-likelihood,-1.00,3.52,1.0,0.06,0.11,0.47
5,neyman,-2.00,3.54,1.0,0.06,0.11,0.47


In [48]:
pg.chi2_independence(subgroupdf,x='cp_0',y='restecg_1')[2].round(2)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00,4.52,1.0,0.03,0.12,0.57
1,cressie-read,0.67,4.52,1.0,0.03,0.12,0.57
2,log-likelihood,0.00,4.53,1.0,0.03,0.12,0.57
3,freeman-tukey,-0.50,4.54,1.0,0.03,0.12,0.57
4,mod-log-likelihood,-1.00,4.55,1.0,0.03,0.12,0.57
5,neyman,-2.00,4.59,1.0,0.03,0.12,0.57



According to the standard Chi-Squared test, we currently only have chi-squared values, Cramer's V values, and power values. Still, according to classical statistics, there is no significant relationship between cp_0 and restecg_0, but we can say there is a significant relationship with restecg_1. However, while doing this, the degrees of freedom (dof) will be 10 due to adjustment, and we can consider that the significant p-value for adjusted alpha will change.  Ultimately, this only indicates the presence of a relationship but cannot specify the ranking or level of it. At this point, this method appears to be effective.
#
Even though the dataset in question did not provide particularly significant findings, my thesis research yielded the outcomes I was hoping for. For instance, in the subgroups of admitting categories, the age group of 0-2 showed a heightened risk for the respiratory failure subgroup (OR: 3.64, Z: 10.32), whereas a lesser association significance was observed for cases of intoxication admissions (OR: 0.15, Z: -6.9). On the other hand, in the age group over 12, a reduced risk was noted for respiratory failure (OR: 0.31, Z: -6.77), but a considerably increased risk was seen for intoxication admissions when compared to other age groups (OR: 4.39, Z: 10.6). These findings intuitively aligned with my medical hypotheses.